In [ ]:
# Importing the libraries

import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.metrics as tfm
from skimage.io import imread
from skimage.transform import resize
from sklearn.metrics import (accuracy_score, classification_report, f1_score,
                             precision_score)
from tensorflow.keras.activations import linear, relu
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [ ]:
# Loading the training dataset

data_dir = '/kaggle/input/bunacare-train'
classes = ['healthy', 'rust', 'spot']
features_array = []
targets_array = []
width, height = 800, 400

for c in classes:
    path = os.path.join(data_dir, c)
    for img in os.listdir(path):
        img_array = imread(os.path.join(path, img))
        img_resized = resize(img_array, (width, height, 3))
        features_array.append(img_resized.flatten())
        targets_array.append(classes.index(c))

In [ ]:
# Loading the testing dataset

test_data_dir = '/kaggle/input/bunacare-test'
test_features_array = []
test_targets_array = []

for c in classes:
    path = os.path.join(test_data_dir, c)
    for img in os.listdir(path):
        img_array = imread(os.path.join(path, img))
        img_resized = resize(img_array, (width, height, 3))
        test_features_array.append(img_resized.flatten())
        test_targets_array.append(classes.index(c))

In [ ]:
# Preprocessing the datasets

features = np.array(features_array)
targets = np.array(targets_array)

test_features = np.array(test_features_array)
test_targets = np.array(test_targets_array)

df = pd.DataFrame(features)
df['targets'] = targets

test_df = pd.DataFrame(test_features)
test_df['targets'] = test_targets

X_train = df.iloc[:, :-1]
y_train = df.iloc[:, -1]

X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1]

In [ ]:
# Peeking at the data

print(X_train.head())
print(y_train.head())

In [ ]:
# Checking dimensions of the data

print ('The shape of X is: ' + str(X_train.shape))
print ('The shape of y is: ' + str(y_train.shape))

In [ ]:
# Defining the model

input_shape = X_train.shape[1]
model = Sequential(
    [
        tf.keras.Input(shape=(input_shape,)),
        Dense(units=81, activation='relu', name='L1'),
        Dense(units=27, activation='relu', name='L2'),
        Dense(units=9, activation='relu', name='L3'),
        Dense(units=3, activation='linear', name='L4')
    ], name='model'
)

model.summary()

In [ ]:
# Training the model

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6)
)

model.fit(X_train, y_train, epochs=30)

In [ ]:
# Evaluating the model

test_predictions = model.predict(X_test)
y_pred = np.argmax(test_predictions, axis=1)

accuracy = accuracy_score(y_test, y_pred)
precision_score(y_test, y_pred, zero_division=0, average='weighted')
f1_score(y_test, y_pred, zero_division=0, average='weighted')
class_report = classification_report(y_test, y_pred)

print('Accuracy: ', accuracy * 100, '%')
print(class_report)

In [ ]:
# Testing random individual images

c = random.randint(0, 2)
n = random.randint(0, 7)

random_img = imread(f'{test_data_dir}/{classes[c]}/{classes[c]}-{str(n).zfill(2)}.jpg')
random_img_resized = resize(random_img, (width, height, 3))

print(f'Actual: {classes[c]}')
plt.imshow(random_img_resized)
plt.show()

predictions = model.predict(random_img_resized.flatten().reshape(1, -1))
print(f'Predicted: {classes[np.argmax(predictions)]}')

In [ ]:
# Saving the model

model.save('model.h5')